# Adaptively Sampled MPC

This is a simple demonstration of an adaptively sampled region quadtree used to approximate the solutions to a model-predictive control problem. 

In [2]:
using RegionTrees
import StaticArrays: SVector
using Plots
Plots.gr()

Plots.GRBackend()

In [15]:
# This module implements our MPC code, which consists of a 1-dimensional
# double-integrator system, forward-euler time-stepping dynamics, and a 
# 10 step prediction window. 

module mpc

using JuMP
using Gurobi
using StaticArrays
using Interpolations
using RegionTrees
import RegionTrees: AbstractRefinery, needs_refinement, refine_data

# Solve the MPC problem from a given initial position and velocity
function run_mpc(q0, v0)
    model = Model(solver=GurobiSolver(OutputFlag=0))
    num_time_steps = 10
    dt = 0.1
    u_limit = 3
    C_q = 100
    c_vfinal = 100
    C_u = 1

    @variable model q[1:num_time_steps]
    @variable model v[1:num_time_steps]
    @variable model u[1:num_time_steps]

    @constraint model [i=2:num_time_steps] q[i] == q[i-1] + v[i-1] * dt
    @constraint model [i=2:num_time_steps] v[i] == v[i-1] + u[i-1] * dt
    @constraint model u .<= u_limit
    @constraint model u .>= -u_limit
    @constraint model q[1] == q0
    @constraint model v[1] == v0

    @objective model Min C_q * sum{q[i]^2, i=1:num_time_steps} + c_vfinal * v[end]^2 + C_u * sum{u[i]^2, i=1:num_time_steps}
    solve(model)

    getvalue(q), getvalue(v), getvalue(u)
end

# The MPCRefinery provides enough behavior to implement the 
# RegionTrees AdaptiveSampling interface, which lets us generate
# a quadtree of initial states and their corresponding MPC solutions.
immutable MPCRefinery <: AbstractRefinery
end

function evaluate(cell, point)
    p = (point - cell.boundary.origin) ./ cell.boundary.widths
    cell.data[p[1] + 1, p[2] + 1]
end

# A cell in the quadtree needs refinement if its interpolated solution
# derived from its vertices is not a good fit for the true MPC solution
# at its center and the center of each of its faces
function needs_refinement(::MPCRefinery, cell)
    for x in body_and_face_centers(cell.boundary)
        value_interp = evaluate(cell, x)
        value_true = run_mpc(x[1], x[2])[3]
        if !isapprox(value_interp[1], value_true[1], rtol=1e-1, atol=1e-1)
            return true
        end
    end
    false
end

# The data element associated with a cell is a bilinear interpolation
# of the MPC function evaluated at the vertices of the cell.
function refine_data(::MPCRefinery, boundary)
    f = v -> run_mpc(v[1], v[2])[3]
    interpolate(f.(vertices(boundary)),
                BSpline(Linear()), OnGrid())
end

# Needed to let us interpolate directly on arrays of control inputs
Base.one{T, N}(::Type{Array{T, N}}) = one(T)

end

mpc

In [16]:
# Simulate a given control function for the double integerator
# model.
function simulate(controller, q0, v0, dt, timespan)
    num_time_steps = timespan / dt
    qs = [q0]
    vs = [v0]
    ts = [0.0]
    q = q0
    v = v0
    for t in 0:dt:(timespan)
        u = controller(t, q, v)
        q += v * dt
        v += u * dt
        push!(qs, q)
        push!(vs, v)
        push!(ts, t)
    end
    ts, qs, vs
end
        

simulate (generic function with 1 method)

In [17]:
controller = (t, q, v) -> begin
    q, v, u = mpc.run_mpc(q, v)
    u[1]
end

(::#5) (generic function with 1 method)

In [18]:
t, q, v = simulate(controller, 10.0, 0.0, 0.1, 10)

([0.0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8  …  9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0],[10.0,10.0,9.97,9.91,9.82,9.7,9.55,9.37,9.16,8.92  …  8.59724e-5,5.36258e-5,2.83979e-5,9.84092e-6,-2.88847e-6,-1.08171e-5,-1.50039e-5,-1.64406e-5,-1.59925e-5,-1.43722e-5],[0.0,-0.3,-0.6,-0.9,-1.2,-1.5,-1.8,-2.1,-2.4,-2.7  …  -0.000323466,-0.000252279,-0.00018557,-0.000127294,-7.9286e-5,-4.18688e-5,-1.43665e-5,4.48084e-6,1.6203e-5,2.23757e-5])

In [19]:
# State space portrait of the solution, starting from q=10, v=0
plot(q, v, xlim=(-10, 10), ylim=(-10, 10), xlabel="q", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 q 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,184.837 596.063,190.264 595.244,195.691 593.605,201.118 591.147,206.545 587.869,211.972 583.772,217.399 578.856,222.826 573.121,228.253 566.566,233.68 
 559.191,239.107 550.998,244.534 541.985,249.961 532.152,255.388 521.501,260.815 510.029,266.242 497.739,271.669 484.629,277.096 470.7,282.523 455.951,287.95 
 440.383,293.377 423.996,289.428 408.205,284.001 393.233,278.574 379.081,273.147 365.748,267.72 353.234,262.293 341.54,256.866 330.665,251.439 320.61,246.012 
 311.374,240.585 302.957,235.158 295.359,229.731 288.581,224.304 282.623,218.877 277.483,213.45 273.164,208.023 269.663,202.596 266.982,197.169 265.12,191.742 
 264.077,186.315 263.854,180.888 264.451,175.461 265.866,170.034 268.101,164.607 271.156,159.18 275.029,153.753 279.723,148.326 285.235,142.899 291.567,140.734 
 298.225,143.026 304.538,148.235 310.064,153.662 314.771,159.089 318.659,164.516 321.727,169.943 323.975,175.37 325.405,180.121 326.117,183.564 326.309,185.879 
 326.152,187.278 325.783,187.973 325.31,188.157 324.809,187.999 324.331,187.634 323.909,187.166 323.557,186.672 323.28,186.202 323.074,185.787 322.931,185.441 
 322.84,185.169 322.789,184.967 322.77,184.826 322.772,184.737 322.787,184.688 322.809,184.669 322.835,184.67 322.86,184.686 322.883,184.708 322.902,184.733 
 322.918,184.758 322.93,184.78 322.939,184.799 322.944,184.815 322.948,184.827 322.949,184.835 322.95,184.841 322.949,184.844 322.948,184.845 322.947,184.846 
 322.945,184.845 322.944,184.844 322.943,184.843 322.942,184.842 322.942,184.84 322.941,184.839 322.941,184.838 322.94,184.838 322.94,184.837 322.94,184.837 
 322.94,184.837 322.94,184.837 
 "/>

In [20]:
plot(t, q, xlabel="t", ylabel="q", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 t 
 
 
 q 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 58.7073,3.93701 58.7073,3.93701 64.0809,4.82936 69.4545,6.61406 74.828,9.29112 80.2016,12.8605 85.5751,17.3223 90.9487,22.6764 96.3222,28.9229 101.696,36.0617 
 107.069,44.0928 112.443,53.0164 117.816,62.8322 123.19,73.5405 128.564,85.141 133.937,97.634 139.311,111.019 144.684,125.297 150.058,140.467 155.431,156.529 
 160.805,173.484 166.178,191.331 171.552,208.529 176.926,224.834 182.299,240.247 187.673,254.767 193.046,268.396 198.42,281.132 203.793,292.975 209.167,303.926 
 214.54,313.985 219.914,323.152 225.288,331.426 230.661,338.808 236.035,345.297 241.408,350.894 246.782,355.599 252.155,359.411 257.529,362.331 262.902,364.359 
 268.276,365.494 273.65,365.737 279.023,365.088 284.397,363.546 289.77,361.112 295.144,357.785 300.517,353.567 305.891,348.456 311.265,342.452 316.638,335.556 
 322.012,328.304 327.385,321.43 332.759,315.411 338.132,310.285 343.506,306.051 348.879,302.71 354.253,300.261 359.627,298.704 365,297.929 370.374,297.719 
 375.747,297.891 381.121,298.292 386.494,298.808 391.868,299.353 397.241,299.873 402.615,300.333 407.989,300.716 413.362,301.018 418.736,301.242 424.109,301.398 
 429.483,301.498 434.856,301.552 440.23,301.574 445.603,301.572 450.977,301.555 456.351,301.531 461.724,301.503 467.098,301.476 472.471,301.451 477.845,301.429 
 483.218,301.412 488.592,301.399 493.965,301.39 499.339,301.384 504.713,301.38 510.086,301.378 515.46,301.378 520.833,301.379 526.207,301.38 531.58,301.381 
 536.954,301.383 542.327,301.384 547.701,301.385 553.075,301.386 558.448,301.387 563.822,301.387 569.195,301.388 574.569,301.388 579.942,301.388 585.316,301.388 
 590.689,301.388 596.063,301.388 
 "/>

In [21]:
plot(t, v, xlabel="t", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 t 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 58.7073,108.471 58.7073,121.334 64.0809,134.197 69.4545,147.061 74.828,159.924 80.2016,172.787 85.5751,185.651 90.9487,198.514 96.3222,211.377 101.696,224.241 
 107.069,237.104 112.443,249.967 117.816,262.831 123.19,275.694 128.564,288.557 133.937,301.421 139.311,314.284 144.684,327.147 150.058,340.01 155.431,352.874 
 160.805,365.737 166.178,356.376 171.552,343.513 176.926,330.65 182.299,317.786 187.673,304.923 193.046,292.06 198.42,279.196 203.793,266.333 209.167,253.47 
 214.54,240.606 219.914,227.743 225.288,214.88 230.661,202.016 236.035,189.153 241.408,176.29 246.782,163.426 252.155,150.563 257.529,137.7 262.902,124.837 
 268.276,111.973 273.65,99.1099 279.023,86.2466 284.397,73.3833 289.77,60.52 295.144,47.6567 300.517,34.7934 305.891,21.93 311.265,9.06674 316.638,3.93701 
 322.012,9.36792 327.385,21.7144 332.759,34.5777 338.132,47.441 343.506,60.3043 348.879,73.1676 354.253,86.031 359.627,97.2934 365,105.453 370.374,110.941 
 375.747,114.257 381.121,115.903 386.494,116.34 391.868,115.965 397.241,115.099 402.615,113.991 407.989,112.821 413.362,111.707 418.736,110.722 424.109,109.903 
 429.483,109.257 434.856,108.778 440.23,108.444 445.603,108.233 450.977,108.117 456.351,108.072 461.724,108.076 467.098,108.112 472.471,108.165 477.845,108.224 
 483.218,108.283 488.592,108.336 493.965,108.382 499.339,108.418 504.713,108.446 510.086,108.466 515.46,108.479 520.833,108.487 526.207,108.491 531.58,108.491 
 536.954,108.49 542.327,108.488 547.701,108.485 553.075,108.482 558.448,108.479 563.822,108.476 569.195,108.474 574.569,108.473 579.942,108.471 585.316,108.471 
 590.689,108.47 596.063,108.47 
 "/>

# Approximating the solutions space
Now that we've written down and tested the true MPC solution, we can try to approximate it. The AdaptiveSampling() function will generate a quadtree by iteratively refining each cell in the space until the solutions to the MPC problem within that cell are well-approximated by the interpolation. 

In [29]:
root = RegionTrees.AdaptiveSampling(mpc.MPCRefinery(), SVector(-10, -10), SVector(20, 20))

Cell: RegionTrees.HyperRectangle{2,Float64}([-10.0,-10.0],[20.0,20.0])

In [37]:
# Now we can plot each region in the quadtree. Note that we end up with
# a lot of detail along the switching surface from maximum acceleration
# to maximum deceleration
plt = plot(xlim=(-10, 10), ylim=(-10, 10), legend=nothing, grid=false)
for cell in RegionTrees.allleaves(root)
    v = hcat(collect(RegionTrees.vertices(cell.boundary))...)
    plot!(v[1,[1,2,4,3,1]], v[2,[1,2,4,3,1]])
end
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10

# Using the Approximate Solution
We can use the quadtree to produce an approximate controller for our system. Given the current state, we can look up the cell in the quadtree which encompasses that state, then use that cell's interpolation to find our control tape. 

In [34]:
approx_controller = (t, q, v) -> begin
    x = [q, v]
    leaf = RegionTrees.findleaf(root, x)
    u = mpc.evaluate(leaf, x)
    u[1]
end

(::#11) (generic function with 1 method)

In [35]:
t, q, v = simulate(approx_controller, 10.0, 0.0, 0.1, 10)

([0.0,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8  …  9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0],[10.0,10.0,9.97,9.91,9.82,9.7,9.55,9.37,9.16,8.92  …  0.000117648,7.38866e-5,3.96448e-5,1.43656e-5,-3.05489e-6,-1.39802e-5,-1.98267e-5,-2.19265e-5,-2.14469e-5,-1.93518e-5],[0.0,-0.3,-0.6,-0.9,-1.2,-1.5,-1.8,-2.1,-2.4,-2.7  …  -0.000437612,-0.000342419,-0.000252791,-0.000174205,-0.000109253,-5.84649e-5,-2.09982e-5,4.7965e-6,2.0951e-5,2.95725e-5])

In [36]:
# The results are similar to, but not quite as good as, the 
# values obtained from the exact MPC solution. 
plot(q, v, xlim=(-10, 10), ylim=(-10, 10), xlabel="q", ylabel="v", legend=nothing)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 q 
 
 
 v 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,184.837 596.063,190.264 595.244,195.691 593.605,201.118 591.147,206.545 587.869,211.972 583.772,217.399 578.856,222.826 573.121,228.253 566.566,233.68 
 559.191,239.107 550.998,244.534 541.985,249.961 532.152,255.388 521.501,260.815 510.029,266.242 497.739,271.669 484.629,277.096 470.7,282.523 455.951,287.95 
 440.383,293.377 423.996,289.67 408.168,284.243 393.16,278.816 378.971,273.389 365.602,267.962 353.052,262.535 341.321,257.108 330.41,251.681 320.318,246.254 
 311.045,240.827 302.592,235.4 294.958,229.973 288.143,224.546 282.148,219.119 276.972,213.692 272.616,208.265 269.079,202.838 266.361,197.411 264.463,191.983 
 263.384,186.556 263.124,181.129 263.684,175.702 265.063,170.275 267.262,164.848 270.28,159.421 274.117,153.994 278.773,148.567 284.249,143.14 290.545,140.554 
 297.23,142.287 303.655,146.672 309.417,151.846 314.398,157.211 318.569,162.638 321.92,168.056 324.454,173.394 326.182,178.43 327.149,182.912 327.44,186.085 
 327.251,188.017 326.771,188.992 326.144,189.272 325.474,189.081 324.833,188.606 324.264,187.986 323.789,187.326 323.413,186.695 323.133,186.136 322.937,185.668 
 322.811,185.299 322.741,185.023 322.713,184.831 322.714,184.708 322.734,184.64 322.763,184.613 322.797,184.614 322.831,184.633 322.862,184.663 322.888,184.696 
 322.909,184.729 322.926,184.76 322.937,184.785 322.945,184.806 322.95,184.822 322.952,184.834 322.952,184.841 322.952,184.846 322.95,184.848 322.949,184.849 
 322.947,184.848 322.945,184.847 322.944,184.845 322.943,184.843 322.942,184.842 322.941,184.84 322.941,184.839 322.94,184.838 322.94,184.837 322.94,184.837 
 322.94,184.837 322.94,184.837 
 "/>